In [3]:
import OrdinaryDiffEq as ODE
import SciMLSensitivity as SMS
import Enzyme
import Enzyme: make_zero
using ComponentArrays

# p 的定义: [θ_s, θ_r, Ks, α, n, dz, Q_rain]
# 对应索引:   1    2    3   4  5  6     7
function richards_1layer(du, u, p, t)
    # --- 1. 解包参数 (为了代码可读性，编译器会优化掉这些临时变量) ---
    # (; θ_s, θ_r, Ks, n, dz, Q_rain) = p
    θ_s   = p[1]
    θ_r   = p[2]
    Ks    = p[3]
    # α   = p[4] # 单层自由排水不需要计算势能梯度，暂时不用 α
    n     = p[5]
    dz    = p[6] # 土层厚度
    Q_in  = p[7] # 顶部入渗 (降雨)
    # Q_in = Q_rain

    θ = u[1]

    # --- 2. 计算水力特性 (Van Genuchten) ---
    # 物理约束：使用 clamp 防止 θ 超出 [θ_r, θ_s] 导致计算错误
    # 注意：Enzyme 支持 Base.clamp
    θ_safe = clamp(θ, θ_r + 1e-5, θ_s - 1e-5)
    
    # 有效饱和度 Se
    Se = (θ_safe - θ_r) / (θ_s - θ_r)
    
    # Mualem 模型计算 K (导水率)
    m = 1.0 - 1.0/n
    # K = Ks * sqrt(Se) * (1 - (1 - Se^(1/m))^m)^2
    term1 = 1.0 - Se^(1.0/m)
    # 为了数值稳定，防止 term1 < 0
    term1 = max(term1, 0.0) 
    term2 = (1.0 - term1^m)^2
    K = Ks * sqrt(Se) * term2

    # --- 3. 质量守恒 (Richards Equation) ---
    # 底部自由排水假设: Q_out = K
    Q_out = K
    
    # dθ/dt = (Q_in - Q_out) / dz
    du[1] = (Q_in - Q_out) / dz
    return nothing
end

# 参数: [θ_s, θ_r, Ks, α, n, dz, Q_rain]
# 真实参数: Ks=10.0, n=2.0
ps0 = (; θ_s = 0.45, θ_r = 0.05, Ks = 10.0, α = 0.01, n=2.0, dz=10.0, Q_rain=2.0)
ps0 = ComponentArray(ps0)

u0 = [0.2] 
tspan = (0.0, 5.0)
prob = ODE.ODEProblem(richards_1layer, u0, tspan, ps0)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
Non-trivial mass matrix: false
timespan: (0.0, 5.0)
u0: 1-element Vector{Float64}:
 0.2

In [4]:
function loss_adjoint_enzyme(u0, p, prob, axes)
    # 关键点：使用 InterpolatingAdjoint 配合 EnzymeVJP
    # 这告诉 SciML：在计算伴随方程的 VJP 时，也请使用 Enzyme，实现全栈加速
    sensealg = SMS.InterpolatingAdjoint(autojacvec=SMS.EnzymeVJP())
    # 使用 remake 更新参数
    ps = ComponentArray(p, axes)
    @show p, ps
    new_prob = ODE.remake(prob, u0=u0, p=p)
    
    # 求解 (降低一点容差方便演示)
    sol = ODE.solve(new_prob, ODE.Tsit5(); saveat=0.1, sensealg=sensealg, abstol=1e-6, reltol=1e-6)
    
    # 目标：假设我们希望最后时刻的含水量尽可能大 (随便定义的一个 loss)
    # 或者简单地 sum(sol) 测试梯度存在性
    return sum(sol)
end

# 准备这一轮要微分的参数 (假设我们猜测 Ks=5.0, n=1.5，看梯度方向)
p_guess = (; θ_s = 0.45, θ_r = 0.05, Ks = 5.0, α = 0.01, n=1.5, dz=10.0, Q_rain=2.0)
p_guess = ComponentArray(p_guess) #|> collect

p_data = getdata(p_guess) 
p_axes = getaxes(p_guess)
# du0 = make_zero(u0) 
# dp  = make_zero(p_data)

loss_enzyme(u0, p) = loss_adjoint_enzyme(u0, p, prob, p_axes)

du0 = make_zero(u0) 
dp  = make_zero(p_data)
println("开始 Enzyme 自动微分...")

# 计时并运行
@time Enzyme.autodiff(
    # Enzyme.Reverse,
    Enzyme.set_runtime_activity(Enzyme.Reverse),
    loss_enzyme,
    Enzyme.Active,
    Enzyme.Duplicated(u0, du0),
    Enzyme.Duplicated(p_data, dp),
    # Enzyme.Const(prob),
    # Enzyme.Const(p_axes)
)

println("\n=== 梯度计算结果 ===")
# p 的索引: [θ_s, θ_r, Ks, α, n, dz, Q_rain]
#             1    2    3   4  5  6     7
println("dLoss / dKs (p[3]): ", dp[3])
println("dLoss / dn  (p[5]): ", dp[5])
println("dLoss / dRain (p[7]): ", dp[7])

# 物理意义检查:
# 如果 Loss = sum(θ)，而 dLoss/dRain > 0，说明降雨越多，总含水量越高 -> 符合物理直觉。
if dp[7] > 0
    println("物理一致性检查通过: 增加降雨会导致含水量增加 (梯度为正)。")
else
    println("物理一致性警告: 梯度方向可能异常，请检查方程。")
end

开始 Enzyme 自动微分...
(p, ps) = ([0.45, 0.05, 5.0, 0.01, 1.5, 10.0, 2.0], (θ_s = 0.45, θ_r = 0.05, Ks = 5.0, α = 0.01, n = 1.5, dz = 10.0, Q_rain = 2.0))
  8.622948 seconds (35.47 M allocations: 1.755 GiB, 3.21% gc time, 99.94% compilation time: 5% of which was recompilation)

=== 梯度计算结果 ===
dLoss / dKs (p[3]): -0.1411489825787846
dLoss / dn  (p[5]): -1.6274689584758448
dLoss / dRain (p[7]): 1.1277204649006547
物理一致性检查通过: 增加降雨会导致含水量增加 (梯度为正)。


In [31]:
p_guess[2]

0.05